In [1]:
# Get Omphalos github repository in path so modules can be loaded in
import sys
sys.path.insert(0,'/Users/thomasdodd/Library/CloudStorage/OneDrive-MillfieldEnterprisesLimited/github/Omphalos')
sys.path.insert(1,'/Users/thomasdodd/Library/CloudStorage/OneDrive-MillfieldEnterprisesLimited/github')

# Import generic data science modules
import numpy as np
import pandas as pd
import xarray as xr

# Import misc
import wandb
import xgboost as xgb
import re
from importlib import reload

# Import Omphalos modules.
from omphalos import file_methods as fm
from omphalos import attributes as attr
from omphalos import labels as lbls
from analysis import helper as hp

# Import plotting tools
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from mpl_toolkits import mplot3d

# Setup random state
random_state = 69
np.random.state = random_state
np.random.seed = random_state

/Users/thomasdodd/opt/anaconda3/envs/omphalos/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Training Data Import & Cleaning

In [2]:
# Define all the attributes we have varied in .yaml file
attribute_names = ["Al+++","Ca++","Fe++","K+","Na+","Mg++","SiO2(aq)","Cl-"]

In [3]:
# Port in and unpack all the runs we need from the .pkl file.
TrainSet_dict = fm.unpickle('/Users/thomasdodd/Library/CloudStorage/OneDrive-MillfieldEnterprisesLimited/Cambridge/AI4ER/Easter/MRes/CrunchFlow_Work/bfm/2022-06-18_bfm_5-5_32000runs/completed_run.pkl')

In [4]:
# Filter all errored files out of the dictionary
dataset_dict, error_dict = hp.filter_errors(TrainSet_dict)

Returned 27509 files without errors out of a total possible 32000.
4491 files had errors.
0 files had unhandled errors.
File failure rate: 16.32556617834163 %.
To see unhandled errors, run with verbose=True.


In [5]:
# Get a df of all the start attributes of interest
attributes_all_df = attr.get_condition(dataset_dict,"f_i_onehundred",species_concs=True)
attributes_all_df = attributes_all_df.loc[:, attribute_names]
attributes_all_df

/Users/thomasdodd/Library/CloudStorage/OneDrive-MillfieldEnterprisesLimited/github/Omphalos/omphalos/attributes.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  species_attrs = species_attrs.append(primary_species(data_set[i], condition), ignore_index=True)


,Al+++,Ca++,Fe++,K+,Na+,Mg++,SiO2(aq),Cl-
0,0.016153,0.016134,0.012228,0.018647,0.016275,0.017318,0.003776,0.016377
1,0.003152,0.013060,0.013292,0.013327,0.012281,0.013464,0.007094,0.002883
2,0.010019,0.018628,0.000623,0.004058,0.019369,0.018086,0.014039,0.009710
3,0.011326,0.012745,0.002625,0.011348,0.000538,0.014791,0.010975,0.012746
4,0.006303,0.010286,0.002111,0.017350,0.012449,0.019303,0.002528,0.018715
...,...,...,...,...,...,...,...,...
27504,0.010794,0.008441,0.018678,0.008334,0.015491,0.010218,0.018763,0.007174
27505,0.015221,0.018727,0.004800,0.005948,0.011802,0.010917,0.014930,0.007451
27506,0.014366,0.008698,0.004591,0.003868,0.000819,0.011543,0.014164,0.005451
27507,0.001577,0.001195,0.003691,0.018730,0.008866,0.004941,0.016075,0.009608


In [6]:
# Get an array of the end carbonate volumes generated
NonCalSidMag_arr = ["Diopside","Diopside_a","Hedenbergite","Hedenbergite_a",
                    "Albite","Albite_a","Anorthite","Anorthite_a","M_Microcline",
                    "M_Microcline_a","M_Microcline_b","Forsterite","Forsterite_a",
                    "Fayalite","Fayalite_a","Antigorite","Antigorite_a","Greenalite",
                    "Greenalite_a","Calcite_a","Siderite_a","Magnesite_a"]
CalSidMag_arr = ["Calcite","Siderite","Magnesite"]

Vols_ds = lbls.raw(dataset_dict, 'volume')
NrXDiscretisedBlocks = len(Vols_ds.X.values)
EndVols_ds = Vols_ds.sel(time=280.0)
CalSidMagEndVols_ds = EndVols_ds.drop(labels=NonCalSidMag_arr)

CalSidMagEndVols_da = CalSidMagEndVols_ds.to_array(dim='arbitrary_array')
CalSidMagEndVols_da = CalSidMagEndVols_da.astype(str)
CalSidMagEndVols_da = CalSidMagEndVols_da.str.replace("^\d+\.\d+-\d+$", "0", regex=True)
CalSidMagEndVols_da = CalSidMagEndVols_da.str.replace("^\d+\.\d+\+\d+$", "5000.0000", regex=True)
CalSidMagEndVols_da = CalSidMagEndVols_da.astype(float)

CalSidMagEndVols_ds = CalSidMagEndVols_da.to_dataset(dim="arbitrary_array")
SpatialSummedCalSidMagEndVols_ds = CalSidMagEndVols_ds.sum(dim=["X","Y","Z"])
SpatialSummedCarbEndVols_ds = SpatialSummedCalSidMagEndVols_ds.assign(Carbonates=lambda SpatialSummedCalSidMagEndVols_ds: SpatialSummedCalSidMagEndVols_ds.Calcite + SpatialSummedCalSidMagEndVols_ds.Siderite + SpatialSummedCalSidMagEndVols_ds.Magnesite)
SpatialSummedCarbEndVols_ds = SpatialSummedCarbEndVols_ds.drop(labels=CalSidMag_arr)
SpatialSummedCarbEndVols_arr = np.array(SpatialSummedCarbEndVols_ds.Carbonates.values)
attributes_all_df["Carbonates_sum"] = SpatialSummedCarbEndVols_arr.tolist()

attributes_cleaned_df = attributes_all_df
attributes_cleaned_df.loc[attributes_cleaned_df['Carbonates_sum'] > (1*NrXDiscretisedBlocks), 'Carbonates_sum'] = np.nan
attributes_cleaned_df.loc[attributes_cleaned_df['Carbonates_sum'] == np.inf, 'Carbonates_sum'] = np.nan
attributes_cleaned_df = attributes_cleaned_df.dropna()
attributes_cleaned_df = attributes_cleaned_df.reset_index(drop=True)
attributes_cleaned_df["Carbonates_avg"] = (attributes_cleaned_df["Carbonates_sum"] / NrXDiscretisedBlocks)
attributes_cleaned_df["Carbonates_pct"] = (attributes_cleaned_df["Carbonates_avg"] / 1) * 100
attributes_cleaned_df = attributes_cleaned_df.drop(['Carbonates_sum', 'Carbonates_avg'], axis=1)
attributes_cleaned_df

,Al+++,Ca++,Fe++,K+,Na+,Mg++,SiO2(aq),Cl-,Carbonates_sum
0,0.016153,0.016134,0.012228,0.018647,0.016275,0.017318,0.003776,0.016377,1.672473
1,0.003152,0.013060,0.013292,0.013327,0.012281,0.013464,0.007094,0.002883,1.876084
2,0.010019,0.018628,0.000623,0.004058,0.019369,0.018086,0.014039,0.009710,1.273991
3,0.011326,0.012745,0.002625,0.011348,0.000538,0.014791,0.010975,0.012746,1.234340
4,0.006303,0.010286,0.002111,0.017350,0.012449,0.019303,0.002528,0.018715,2.290114
...,...,...,...,...,...,...,...,...,...
27504,0.010794,0.008441,0.018678,0.008334,0.015491,0.010218,0.018763,0.007174,1.115855
27505,0.015221,0.018727,0.004800,0.005948,0.011802,0.010917,0.014930,0.007451,1.057754
27506,0.014366,0.008698,0.004591,0.003868,0.000819,0.011543,0.014164,0.005451,1.090099
27507,0.001577,0.001195,0.003691,0.018730,0.008866,0.004941,0.016075,0.009608,2.235406


# XGBoost Model Training

In [8]:
# Split the ML datasets into x's (predictor arrays) and y's (target arrays).
x = attributes_cleaned_df.loc[:, attribute_names].astype(float)
y = attributes_cleaned_df["Carbonates_pct"].values.astype(float)

# Generate a numpy matrix from a pandas dataframe.
x = x.to_numpy()

# Normalisation array for each of the columns in x.
x_norms = []
for _ in attribute_names:
    max_val = max(attributes_cleaned_df[_].values)
    x_norms.append(max_val)
    print(f"Normalisation Factor for {_} = {max_val}")

# Normalisation of the np matrix using the x_norm array.
for i, norm in enumerate(x_norms):
    x[:, i] = x[:, i]/norm

# Generate a vertical array of target values.
y = y.reshape(-1,1)

# Normalisation of the target array.
y_norm = max(y)
print(f"Normalisation Factor for Carbonate Volume Generated = {y_norm}")
y = y / y_norm

# Splitting the data into test and train batches using scikit train_test_split method.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=random_state)

Normalisation Factor for Al+++ = 0.0199996777771118
Normalisation Factor for Ca++ = 0.0199998101672167
Normalisation Factor for Fe++ = 0.019999719657439
Normalisation Factor for K+ = 0.019999945896381
Normalisation Factor for Na+ = 0.0199997455006281
Normalisation Factor for Mg++ = 0.0199994907051103
Normalisation Factor for SiO2(aq) = 0.0199990220873342
Normalisation Factor for Cl- = 0.0199997402413945
Normalisation Factor for Carbonate Volume Generated = [96.2359084]


In [9]:
%%time

xgb_defaults = {
    'tree_method': 'hist',
    'gamma': 0,
    'mds': 0,
    'eta': 0.01,
    'l1' : 0,
    'l2' : 1,
    'max_depth' : 0,
    'max_leaves': 6,
    'objective': 'reg:squarederror',
    'max_bin': 10000,
    'grow_policy': 'lossguide'
}

# Initialise the wandb instance
wandb.init(config=xgb_defaults, project='bfm_5-5')

# Setup xgb matrices; one for training and one testing
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

evallist = [(dtest, 'eval'), (dtrain, 'train')]

params = {'max_depth': wandb.config.max_depth,
            'tree_method': wandb.config.tree_method,
            'max_delta_step': wandb.config.mds,
            'eta': wandb.config.eta,
            'objective': wandb.config.objective,
            'alpha': wandb.config.l1,
            'lambda': wandb.config.l2,
            'gamma': wandb.config.gamma,
            'max_leaves': wandb.config.max_leaves,
            'max_bin': wandb.config.max_bin,
            'grow_policy': wandb.config.grow_policy
        }

results = {}

num_round = 300000
bst = xgb.train(params, dtrain, num_round, evallist, evals_result=results, verbose_eval=False, callbacks=[wandb.xgboost.WandbCallback()])
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tjhd97. Use `wandb login --relogin` to force relogin


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-rmse,▁▁▁▂▂▃▃▃▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████
train-rmse,██▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
epoch,299999


CPU times: user 3h 9min 33s, sys: 9min 11s, total: 3h 18min 44s
Wall time: 19min 36s


In [10]:
bst.save_model("bfm_XGBModel_5-5_Epo-300000.json")